In [1]:
import tensorflow as tf 
import numpy as np 
from tensorflow import keras 
import os 
import math 
import random 
import pickle 
import matplotlib.pyplot as plt 
from collections import deque 
from tensorflow.keras import layers
import time 

from vehicle_model_DDPG9_1 import Environment 
from cell_model import CellModel 

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
drving_cycle = '../../OC_SIM_DB/OC_SIM_DB_Cycles/Highway/01_FTP72_fuds.mat'
battery_path = "../../OC_SIM_DB/OC_SIM_DB_Bat/OC_SIM_DB_Bat_e-4wd_Battery.mat"
motor_path = "../../OC_SIM_DB/OC_SIM_DB_Mot/OC_SIM_DB_Mot_id_75_110_Westinghouse.mat"
cell_model = CellModel()
env = Environment(cell_model, drving_cycle, battery_path, motor_path, 10)

num_states = 4

In [3]:
class OUActionNoise: 
    def __init__(self, mean, std_deviation, theta=0.15, dt=1e-2, x_initial=None): 
        self.theta = theta 
        self.mean = mean 
        self.std_dev = std_deviation 
        self.dt = dt 
        self.x_initial = x_initial 
        self.reset() 
        
    def reset(self): 
        if self.x_initial is not None: 
            self.x_prev = self.x_initial 
        else: 
            self.x_prev = 0 
            
    def __call__(self): 
        x = (
             self.x_prev + self.theta * (self.mean - self.x_prev) * self.dt 
            + self.std_dev * np.sqrt(self.dt) * np.random.normal() 
        )
        self.x_prev = x 
        return x 

In [4]:
class Buffer: 
    def __init__(self, stack_size, buffer_capacity=100000, batch_size=64): 
        self.power_mean = 0 
        self.power_std = 0
        self.sum = 0 
        self.sum_deviation = 0 
        self.N = 0 
        
        self.buffer_capacity = buffer_capacity 
        self.batch_size = batch_size 
        self.buffer_counter = 0 
        
        self.state_buffer = np.zeros((self.buffer_capacity, stack_size, num_states))
        self.action_buffer = np.zeros((self.buffer_capacity, 1))
        self.reward_buffer = np.zeros((self.buffer_capacity, 1))
        self.next_state_buffer = np.zeros((self.buffer_capacity, stack_size, num_states))
#         self.terminal_buffer = np.zeros((self.buffer_capacity, 1))
        
    def record(self, obs_tuple):
        self.N += 1 
        index = self.buffer_counter % self.buffer_capacity 
        power = obs_tuple[0][-1][0] 
        
        self.sum += power 
        self.power_mean = self.sum / self.N 
        self.sum_deviation += (power - self.power_mean) ** 2  
        self.power_std = np.sqrt(self.sum_deviation / self.N) 
            
        self.state_buffer[index] = obs_tuple[0]
        self.action_buffer[index] = obs_tuple[1]
        self.reward_buffer[index] = obs_tuple[2]
        self.next_state_buffer[index] = obs_tuple[3]
#         self.terminal_buffer[index] = obs_tuple[4]
        
        self.buffer_counter += 1 
        
    def learn(self): 
        record_range = min(self.buffer_counter, self.buffer_capacity)
        batch_indices = np.random.choice(record_range, self.batch_size)
        
        state_batch = self.state_buffer[batch_indices]
        power_batch = (state_batch[:, :, 0] - self.power_mean) / self.power_std
        state_batch[:, :, 0] = power_batch 
        
        next_state_batch = self.next_state_buffer[batch_indices]
        power_batch = (next_state_batch[:, :, 0] - self.power_mean) / self.power_std
        next_state_batch[:, :, 0] = power_batch 
#         print(state_batch)
        
        state_batch = tf.convert_to_tensor(state_batch)
        action_batch = tf.convert_to_tensor(self.action_buffer[batch_indices])
        reward_batch = tf.convert_to_tensor(self.reward_buffer[batch_indices])
        reward_batch = tf.cast(reward_batch, dtype=tf.float32)
        next_state_batch = tf.convert_to_tensor(next_state_batch)
#         terminal_batch = tf.convert_to_tensor(self.terminal_buffer[batch_indices])
#         terminal_batch = tf.cast(terminal_batch, dtype=tf.float32)
        
        with tf.GradientTape() as tape: 
            target_actions = target_actor(next_state_batch)
            y = reward_batch + gamma * target_critic([next_state_batch, target_actions])
            critic_value = critic_model([state_batch, action_batch])
            critic_loss = tf.math.reduce_mean(tf.square(y - critic_value)) 
        critic_grad = tape.gradient(critic_loss, critic_model.trainable_variables) 
        critic_optimizer.apply_gradients(
            zip(critic_grad, critic_model.trainable_variables)
        )
        
        with tf.GradientTape() as tape: 
            actions = actor_model(state_batch)
            critic_value = critic_model([state_batch, actions])
            actor_loss = - tf.math.reduce_mean(critic_value)
        actor_grad = tape.gradient(actor_loss, actor_model.trainable_variables) 
        actor_optimizer.apply_gradients(
            zip(actor_grad, actor_model.trainable_variables)
        )
        

In [5]:
def update_target(tau): 
    new_weights = [] 
    target_variables = target_critic.weights
    for i, variable in enumerate(critic_model.weights): 
        new_weights.append(target_variables[i] * (1 - tau) + tau * variable)
    target_critic.set_weights(new_weights)
    
    new_weights = [] 
    target_variables = target_actor.weights
    for i, variable in enumerate(actor_model.weights): 
        new_weights.append(target_variables[i] * (1 - tau) + tau * variable)
    target_actor.set_weights(new_weights)
    

In [6]:
def get_actor(stack_size): 
    last_init = tf.random_uniform_initializer(minval=-0.003, maxval=0.003)
    
    inputs = layers.Input(shape=(stack_size, num_states))
    inputs_flatten = layers.Flatten()(inputs) 
    
    out = layers.Dense(512, activation="relu")(inputs_flatten)
#     out = layers.BatchNormalization()(out)
    out = layers.Dense(512, activation="relu")(out)
#     out = layers.BatchNormalization()(out)
    outputs = layers.Dense(1, activation="sigmoid", 
                          kernel_initializer=last_init)(out)
    model = tf.keras.Model(inputs, outputs)
    return model

In [7]:
def get_critic(stack_size): 
    state_input = layers.Input(shape=(stack_size, num_states))
    state_input_flatten = layers.Flatten()(state_input)
    
    state_out = layers.Dense(16, activation="relu")(state_input_flatten)
#     state_out = layers.BatchNormalization()(state_out)
    state_out = layers.Dense(32, activation="relu")(state_out)
#     state_out = layers.BatchNormalization()(state_out)
    
    action_input = layers.Input(shape=(1))
    action_out = layers.Dense(32, activation="relu")(action_input)
#     action_out = layers.BatchNormalization()(action_out)
    
    concat = layers.Concatenate()([state_out, action_out]) 
    
    out = layers.Dense(512, activation="relu")(concat)
#     out = layers.BatchNormalization()(out)
    out = layers.Dense(512, activation="relu")(out)
#     out = layers.BatchNormalization()(out)
    outputs = layers.Dense(1)(out)
    
    model = tf.keras.Model([state_input, action_input], outputs)
    return model 
    

In [8]:
def stack_states(stacked_states, state, stack_size, is_new_episode): 
    if is_new_episode: 
        stacked_states = deque([[0.0] * num_states for _ in range(stack_size)], 
                               maxlen=stack_size)
        for _ in range(stack_size): 
            stacked_states.append(state)
        stacked_array = np.array(stacked_states)
    else: 
        stacked_states.append(state)
        stacked_array = np.array(stacked_states)
    return stacked_array, stacked_states 

In [9]:
def policy(state, noise_object): 
    j_min = state[0][2].numpy()
    j_max = state[0][3].numpy()
    sampled_action = tf.squeeze(actor_model(state)) 
    noise = noise_object()
    sampled_action = sampled_action.numpy() + noise 
    legal_action = sampled_action * j_max 
    legal_action = np.clip(legal_action, j_min, j_max)
#     print(j_min, j_max, legal_action, noise)
    return legal_action 
    

In [10]:
def policy_epsilon_greedy(state, eps): 
    j_min = state[0][-1][-2].numpy()
    j_max = state[0][-1][-1].numpy()
    
    if random.random() < eps: 
        a = random.randint(0, 9)
        return np.linspace(j_min, j_max, 10)[a]
    else: 
        sampled_action = tf.squeeze(actor_model(state)).numpy()  
        legal_action = sampled_action * j_max 
        legal_action = np.clip(legal_action, j_min, j_max)
        return legal_action

In [11]:
std_dev = 0.2 
ou_noise = OUActionNoise(mean=0, std_deviation=0.2)

# actor_model = get_actor() 
# critic_model = get_critic() 

# target_actor = get_actor() 
# target_critic = get_critic() 
# target_actor.set_weights(actor_model.get_weights())
# target_critic.set_weights(critic_model.get_weights())

critic_lr = 0.0005 
actor_lr = 0.00025 
critic_optimizer = tf.keras.optimizers.Adam(critic_lr)
actor_optimizer = tf.keras.optimizers.Adam(actor_lr)

total_episodes = 100
gamma = 0.0 
tau = 0.001 

MAX_EPSILON = 1 
MIN_EPSILON = 0.01 
DECAY_RATE = 0.00002
BATCH_SIZE = 32 
DELAY_TRAINING = 3000 

In [12]:
def initialization(stack_size): 
    actor_model = get_actor(stack_size) 
    critic_model = get_critic(stack_size) 

    target_actor = get_actor(stack_size) 
    target_critic = get_critic(stack_size) 
    target_actor.set_weights(actor_model.get_weights())
    target_critic.set_weights(critic_model.get_weights())
    
    buffer = Buffer(stack_size, 500000, BATCH_SIZE)
    return actor_model, critic_model, target_actor, target_critic, buffer

In [ ]:
print(env.version)

num_trials = 5 
stack_size = 1 
results_dict = {} 
for trial in range(num_trials): 
    actor_model, critic_model, target_actor, target_critic, buffer = initialization(stack_size)
    
    eps = MAX_EPSILON 
    steps = 0

    episode_rewards = [] 
    episode_SOCs = [] 
    episode_FCs = [] 
    
    stacked_states = deque([[0.0] * num_states for _ in range(stack_size)], 
                            maxlen=stack_size)
    for ep in range(total_episodes): 
        start = time.time() 
        episodic_reward = 0 
        
        state = env.reset() 
        state, stacked_states = stack_states(stacked_states, state, stack_size, True) 
#         print(state.shape)     
        while True: 
            tf_state = tf.expand_dims(tf.convert_to_tensor(state), 0)
            action = policy_epsilon_greedy(tf_state, eps)
    #         print(action)
            next_state, reward, done = env.step(action)
            if done: 
                next_state = [0] * num_states 
            
            next_state, stacked_states = stack_states(stacked_states, next_state, stack_size, 
                                                        False)
            buffer.record((state, action, reward, next_state))
            episodic_reward += reward 

            if steps > DELAY_TRAINING: 
                buffer.learn() 
                update_target(tau)
                eps = MIN_EPSILON + (MAX_EPSILON - MIN_EPSILON) * np.exp(-DECAY_RATE * steps)

            steps += 1

            if done: 
                break 

            state = next_state 

        elapsed_time = time.time() - start 
        print("elapsed_time: {:.3f}".format(elapsed_time))
        episode_rewards.append(episodic_reward) 
        episode_SOCs.append(env.SOC)
        episode_FCs.append(env.fuel_consumption) 

    #     print("Episode * {} * Avg Reward is ==> {}".format(ep, avg_reward))
        SOC_deviation_history = np.sum(np.abs(np.array(env.history["SOC"]) - 0.6)) 
        print(
              'Episode: {}'.format(ep + 1),
              "Exploration P: {:.4f}".format(eps),
              'Total reward: {}'.format(episodic_reward), 
              "SOC: {:.4f}".format(env.SOC), 
              "Cumulative_SOC_deviation: {:.4f}".format(SOC_deviation_history), 
              "Fuel Consumption: {:.4f}".format(env.fuel_consumption), 
              "Power_mean: {:.4f}, Power_std: {:.4f}".format(buffer.power_mean, buffer.power_std)
        )

    
    results_dict[trial+1] = {
        "rewards": episode_rewards, 
        "SOCs": episode_SOCs, 
        "FCs": episode_FCs
    }

1
maximum steps, simulation is done ... 
elapsed_time: 18.016
Episode: 1 Exploration P: 1.0000 Total reward: -1002.0178592063007 SOC: 0.8036 Cumulative_SOC_deviation: 94.0756 Fuel Consumption: 61.2617 Power_mean: 2.1068, Power_std: 5.0094
maximum steps, simulation is done ... 
elapsed_time: 17.698
Episode: 2 Exploration P: 1.0000 Total reward: -1010.086383595006 SOC: 0.8088 Cumulative_SOC_deviation: 94.8367 Fuel Consumption: 61.7196 Power_mean: 2.1068, Power_std: 5.0131

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this laye

maximum steps, simulation is done ... 
elapsed_time: 76.452
Episode: 24 Exploration P: 0.5222 Total reward: -525.2870873746907 SOC: 0.6659 Cumulative_SOC_deviation: 47.5047 Fuel Consumption: 50.2406 Power_mean: 2.1068, Power_std: 5.0175
maximum steps, simulation is done ... 
elapsed_time: 76.721
Episode: 25 Exploration P: 0.5083 Total reward: -575.3260303094206 SOC: 0.6803 Cumulative_SOC_deviation: 52.3897 Fuel Consumption: 51.4286 Power_mean: 2.1068, Power_std: 5.0175
maximum steps, simulation is done ... 
elapsed_time: 76.662
Episode: 26 Exploration P: 0.4948 Total reward: -470.28849366085393 SOC: 0.6627 Cumulative_SOC_deviation: 42.0434 Fuel Consumption: 49.8544 Power_mean: 2.1068, Power_std: 5.0175
maximum steps, simulation is done ... 
elapsed_time: 77.321
Episode: 27 Exploration P: 0.4817 Total reward: -475.0343235422195 SOC: 0.6570 Cumulative_SOC_deviation: 42.5552 Fuel Consumption: 49.4820 Power_mean: 2.1068, Power_std: 5.0175
maximum steps, simulation is done ... 
elapsed_time

In [ ]:
results_dict

In [ ]:
with open("DDPG9_1.pkl", "wb") as f: 
    pickle.dump(results_dict, f, pickle.HIGHEST_PROTOCOL)